On importe la bibliothèque TKinter, utilisée pour le moteur graphique.

In [9]:
import tkinter as tk

On configure la grille (GRID) du jeu avec des murs (1) et du sol (0). On indique aussi la position des robots et de la croix (l'objectif).

In [10]:
GRID = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
] # 0 = floor ; 1 = wall

ROBOT_POS = [((7,9), True), ((1, 6), False)]
CROSS_POS = (4,6)

On indique l'alphabet, et la position du dossier ainsi que du fichier où l'on programme les instructions du jeu.


In [11]:
ALPHABET = 'abcdefghijklmnopqrstuvwxyz'

from os import path
import empty as file

MAIN_DIR = path.dirname(file.__file__) # finds the directory (folder) in which is the file 'empty.py'
ANSWER_FILE = path.join(MAIN_DIR, 'answer.txt')

On ajoute les paramètres graphiques: les couleurs, les représentations des objets, et la police.

In [12]:
COLORS = {
    'green': "GREEN",
    'lightgreen': "LIGHTGREEN",
    'darkgreen': "DARKGREEN",
    'purple': "MAGENTA",
    'black': "BLACK",
    'white': "WHITE",
    'lightyellow': "LIGHTYELLOW"
}

SPRITES = {
    'wall': [COLORS['lightgreen']],
    'floor': (COLORS['darkgreen'], COLORS['green']),
    'cross': ('✕', COLORS['red']),
    'robot': ('△', COLORS['yellow']),
    'robot_main': ('△', COLORS['purple']),
    'robot_chosen': ('▲', COLORS['yellow']),
    'robot_main_chosen': ('▲', COLORS['purple'])
}

FONT = 'Segoe UI Black bold'

On crée les classes Mur et Sol. On crée aussi la classe Croix
Ils ont tous trois les attributs:
    pos, leurs coordonées (x, y)
    sprite, leur affichage: une couleur pour les classes mur et sol, et un charactère et une couleur pour la croix

Les classes mur et sol ont aussi l'attribut:
    is_wall, si ils sont un mur, ou non.


In [13]:
class Wall: 
    def __init__(self, pos:tuple) -> None:
        self.pos = pos
        self.sprite = SPRITES['wall']
        self.is_wall = True


class Floor:
    def __init__(self, pos:tuple) -> None:
        self.pos = pos
        self.sprite = SPRITES['floor']
        self.is_wall = False
        
class Cross:
    def __init__(self, pos:tuple) -> None:
        self.pos = pos
        self.sprite = SPRITES['cross']

On crée la classe Robot, on définit les attributs de la classe, et on crée les méthodes pour les activer et désactiver.
Ses attributs sont:
    pos, sa position (x, y)
    is_main, si il est le robot principal (qui doit aller sur la croix)
    cross, la croix
    active, si c'est le robot actif (qui va se déplacer)
    robots, la liste des autres robots

In [14]:
class Robot:
    def __init__(self, pos: tuple, main: bool, grid:list, cross:Cross) -> None:
        self.pos = pos
        self.is_main = main
        self.grid = grid
        self.cross = cross
        self.active = False
        self.setNonActive()
        self.robots:list[Robot] = []
    
    def provideRobots(self, robots:list) -> None:
        self.robots = robots
       
    def setActive(self) -> None:
        self.active = True
        if self.is_main:
            self.sprite = SPRITES['robot_main_chosen']
        else:
            self.sprite = SPRITES['robot_chosen']
        
    def setNonActive(self) -> None:
        self.active = False
        if self.is_main:
            self.sprite = SPRITES['robot_main']
        else:
            self.sprite = SPRITES['robot']
    
    def checkPath(self, path_start:tuple, path_end:tuple) -> bool:
        for robot in self.robots:
            if robot.pos == path_end:
                return False        
        
        if path_end[0] == path_start[0]:
            if path_start[1] < path_end[1]:
                for i in range(path_start[1]+1, path_end[1]+1):
                    if self.grid[i][path_start[0]].is_wall:
                        return False
                robot_block = False
                for robot in self.robots:
                    if robot.pos == (path_end[0], path_end[1]+1):
                        robot_block = True
                        break
                if not (self.grid[path_end[1]+1][path_start[0]].is_wall or robot_block):
                    return False
            else:
                for i in range(path_end[1], path_start[1]):
                    if self.grid[i][path_start[0]].is_wall:
                        return False
                robot_block = False
                for robot in self.robots:
                    if robot.pos == (path_end[0], path_end[1]-1):
                        robot_block = True
                        break
                if not (self.grid[path_end[1]-1][path_start[0]].is_wall or robot_block):
                    return False
            
        elif path_end[1] == path_start[1]:
            if path_start[0] < path_end[0]:
                for i in range(path_start[0]+1, path_end[0]+1):
                    if self.grid[path_start[1]][i].is_wall:
                        return False
                robot_block = False
                for robot in self.robots:
                    if robot.pos == (path_end[0]+1, path_end[1]):
                        robot_block = True
                        break
                if not (self.grid[path_start[1]][path_end[0]+1].is_wall or robot_block):
                    return False
            else:
                for i in range(path_end[0], path_start[0]):
                    if self.grid[path_start[1]][i].is_wall:
                        return False
                robot_block = False
                for robot in self.robots:
                    if robot.pos == (path_end[0]-1, path_end[1]):
                        robot_block = True
                        break
                if not (self.grid[path_start[1]][path_end[0]-1].is_wall or robot_block):
                    return False
                        
        else:
            return False
        
        return True
    
            
    def move(self, destination_pos) -> bool: # success
        path_is_valid = self.checkPath(self.pos, destination_pos)
        if not path_is_valid:
            return False
        self.pos = destination_pos
        return True

On ajoute aussi les mécaniques de mouvement du robot, avec:
une méthode pour vérifier que le mouvement est possible;
une méthode pour se déplacer.

On ajoute le moteur graphique sous la forme d'une classe Game.
On lui définit des attributs et on crée la grille depuis le plan créé plus haut.
La classe Game a pour attributs:
    title, le titre de l'application
    win, si le robot principal est arrivé sur la croix
    grid, la grille de murs et de sols
    robots, la liste de robots
    active_robot, l'index du robot actif dans la liste robots
    moves, le nombre d'instructions effectuées
    file, le fichier answer.txt
    instructions, la liste des instructions, dans l'ordre
    current_instructions, l'index de l'instruction actuelle dans la liste instruction.

On crée des méthodes pour initialiser le jeu:
    __init__() pour construire la classe
    create_grid() pour construire un tableau de mur et de sol.

In [15]:
class Game(tk.Tk):
    def __init__(self, cross_pos:tuple, robots:list[tuple]) -> None:
        super().__init__(screenName='Ricochet Robot')
        self.title = 'Ricochet Robot'
        self.win = False
        self.grid:list[list[Floor | Wall]] = self.create_grid()
        self.cross = Cross(cross_pos)
        self.robots:list[Robot] = []
        self.active_robot = -1
        for i, robot in enumerate(robots):
            self.robots.append(Robot(robot[0], robot[1], self.grid, self.cross))
            if robot[1]:
                self.robots[-1].setActive()
                self.active_robot = i
        for robot in self.robots:
            robot.provideRobots(self.robots)
        self.moves = 0
        self.file = ANSWER_FILE
        self.instructions = []
        self.current_instruction = 0
        file = open(self.file, 'r')
        for line in file:
            self.instructions.append(line.split('\n')[0])
        self.instructions.append((""))
        file.close()
        self.initGrid()
        self.drawGrid()
        self.bind('<space>', lambda event: self.update())
             
    def create_grid(self) -> list:
        grid:list[list] = []
        for y, row in enumerate(GRID):
            grid.append([])
            for x, column in enumerate(row):
                if column == 1:
                    grid[y].append(Wall((x, y)))
                else:
                    grid[y].append(Floor((x, y)))
        return grid
    
    def initGrid(self):
        self.label_grid:list[list[tk.Label]] = []
        for n_row, row in enumerate(self.grid):
            self.label_grid.append([])
            for n_column, element in enumerate(row):
                reference = self.grid[n_row][n_column]
                if len(reference.sprite) > 1:
                    index_color = (n_column + n_row) % len(reference.sprite)
                else: index_color = 0
                self.label_grid[n_row].append(tk.Label(self, background=reference.sprite[index_color], font=(FONT, 30)))
                self.label_grid[n_row][n_column].grid(column=n_column+1, row=n_row+1, sticky='nsew')
        tk.Label(self, background=COLORS['black']).grid(column=0, row=0, sticky='nsew')
        for row in range(len(self.grid)):
            tk.Label(self, background=COLORS['black'], foreground=COLORS['white'], font=(FONT, 22), text=str(row+1)).grid(column=0, row=row+1, sticky='nsew', ipadx=8)
        for col in range(len(self.grid[0])):
            tk.Label(self, background=COLORS['black'], foreground=COLORS['white'], font=(FONT, 22), text=ALPHABET[col].upper()).grid(column=col+1, row=0, sticky='nsew', ipady=8)
        
        self.drawGrid()
                
    def drawGrid(self) -> None:
        for n_row, row in enumerate(self.label_grid):
            for n_column, element in enumerate(row):
                label = ''
                for robot in self.robots:
                    if robot.pos == (n_column, n_row):
                        label = robot.sprite[0]
                        label_color = robot.sprite[1]
                if label == '':
                    if self.cross.pos == (n_column, n_row):
                        label = self.cross.sprite[0]
                        label_color = self.cross.sprite[1]
                if label != '':
                    element.config(text=f'{label}', foreground=label_color)
                else:
                    element.config(text=f'    ')
                    
                element.grid_configure(column=n_column+1, row=n_row+1, sticky='nsew')
             
    def letterToCoordinate(self, letter) -> int:
        for i, l in enumerate(ALPHABET):
            if l == letter:
                return i
        return -1
    
    def getAction(self) -> str:
        return self.instructions[self.current_instruction]
    
    def update(self) -> None:       
        action = self.getAction()
        if "quit" in action or action == "":
            self.quit()
            self.destroy()
            return
        else:
            if action == "s": action = "switch"
            print('')
            print(f'Instruction: {action}')
            self.current_instruction += 1
        if "switch" in action:
            self.switchRobot()
            self.moves += 1
        else:
            try:
                coordinates = (self.letterToCoordinate(action[0]), int(action[1:])-1)
                if not self.robots[self.active_robot].move(coordinates):
                    print("This move is illegal.")
                else:
                    self.moves += 1
            except:
                print("Bad input, try again.")
        self.drawGrid()
        if self.robots[self.active_robot].is_main and self.robots[self.active_robot].pos == self.cross.pos:
            self.win = True
            print(f'You won in {self.moves} moves.')
      
    def switchRobot(self) -> None:
        self.robots[self.active_robot].setNonActive()
        self.active_robot = self.active_robot + 1
        while not self.active_robot < len(self.robots):
            self.active_robot -= len(self.robots)
        self.robots[self.active_robot].setActive()
         

On ajoute les méthodes pour les graphismes:
    init_grid() pour créer les graphismes de la grille
    draw_grid() pour afficher la grille et la mettre à jour

Et on crée les méthodes pour faire fonctionner le jeu:
    getAction() pour choisir l'instruction suivante
    letterToCoordinate() pour transformer des coordonées alphanumériques (ex: e7) en coordonées numériques (ex: (6, 8))
    switchRobot() pour changer de robot actif
    update() pour récupérer et effectuer l'instruction.

Et on lance le jeu.

In [16]:
game = Game(CROSS_POS, ROBOT_POS)
game.mainloop()


Instruction: h2

Instruction: switch

Instruction: b2

Instruction: g2

Instruction: switch

Instruction: i2

Instruction: i9

Instruction: b9

Instruction: b2

Instruction: f2

Instruction: f7

Instruction: e7
You won in 12 moves.
